In [4]:
import os
import cv2  # OpenCV for image processing
from concurrent.futures import ProcessPoolExecutor, as_completed

# Define paths
dataset_path = '/home/research/Kilsar_Sentinal/Data1/Threat Detection/train'  # Update this with your dataset path
images_path = os.path.join(dataset_path, 'images')
labels_path = os.path.join(dataset_path, 'labels')

# Define output folder paths
output_class_0_folder = '/home/research/Kilsar_Sentinal/EfficientNet data/HandGuns'  # Update this with your desired output path for class 0
output_class_1_folder = '/home/research/Kilsar_Sentinal/EfficientNet data/BigGuns'  # Update this with your desired output path for class 1

# Create output folders
os.makedirs(output_class_0_folder, exist_ok=True)
os.makedirs(output_class_1_folder, exist_ok=True)

def process_label_file(label_file):
    """Process a single label file to crop and save gun images."""
    label_path = os.path.join(labels_path, label_file)
    with open(label_path, 'r') as file:
        lines = file.readlines()

    # Get the corresponding image file
    image_file = label_file.replace('.txt', '.jpg')  # Assuming jpg, change if necessary
    image_path = os.path.join(images_path, image_file)

    # Load the image
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    # Initialize flags to check if each gun class is present
    class_0_present = False
    class_1_present = False

    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.split())
        class_id = int(class_id)  # Convert class ID to integer

        # Only consider class 0 (hand gun) and class 1 (big gun)
        if class_id in [0, 1]:
            # Convert YOLO format to bounding box coordinates
            x_center = int(x_center * w)
            y_center = int(y_center * h)
            width = int(width * w)
            height = int(height * h)

            # Calculate the top-left corner of the bounding box
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)

            # Crop the image
            crop = image[y1:y2, x1:x2]

            # Save the cropped image to the corresponding folder
            if class_id == 0:
                cropped_image_filename = f"{os.path.splitext(image_file)[0]}_class_0.jpg"
                cropped_image_path = os.path.join(output_class_0_folder, cropped_image_filename)
                cv2.imwrite(cropped_image_path, crop)
                class_0_present = True

            elif class_id == 1:
                cropped_image_filename = f"{os.path.splitext(image_file)[0]}_class_1.jpg"
                cropped_image_path = os.path.join(output_class_1_folder, cropped_image_filename)
                cv2.imwrite(cropped_image_path, crop)
                class_1_present = True

    return class_0_present, class_1_present

# Use ProcessPoolExecutor to process files in parallel
with ProcessPoolExecutor(max_workers=60) as executor:
    # Submit all label files for processing
    futures = {executor.submit(process_label_file, label_file): label_file for label_file in os.listdir(labels_path) if label_file.endswith('.txt')}
    
    for future in as_completed(futures):
        label_file = futures[future]
        try:
            class_0_found, class_1_found = future.result()
            if class_0_found or class_1_found:
                print(f"Processed {label_file}: Cropped images saved.")
            else:
                print(f"No guns found in {label_file}.")
        except Exception as e:
            print(f"Error processing {label_file}: {e}")

print("Cropping and separating of gun images completed!")


Processed 99d79c30-5cc3-43dc-9127-9be1a022f026_jpg.rf.9389d8e571b147bd257b3e47eea384de.txt: Cropped images saved.
Processed Enfield_Enforcer_jpg.rf.2fc12e84d5ffda8ffedc2ccb391cc20c.txt: Cropped images saved.
Processed ia_100000694_jpg.rf.ce17e253abb43941d48722d6c4422fe8.txt: Cropped images saved.
Processed Direct_Hit-Glock-4_jpg.rf.d374d3909701d92fcabd73b34b230a8a.txt: Cropped images saved.
Processed 9f9ec443-b675-4df2-8097-c455c8cc6887_jpg.rf.a32ca609b417b72fde1c84434d47860c.txt: Cropped images saved.
Processed Martinisporter_jpg.rf.2c6f49a009767e26299f5dc73732be8c.txt: Cropped images saved.
Processed TRCoLUSPMatch3_jpg.rf.a1852ab4826b5c8c020961d5545021c3.txt: Cropped images saved.
Processed River_of_Death-P08-1_jpg.rf.23e43ed6c39cb692389300537525dbf9.txt: Cropped images saved.
Processed a4a5e138-0b8a-4d05-b88d-b7737b37a4e4_jpg.rf.0c55dbed65c64055e41aaff64218eb5b.txt: Cropped images saved.
Processed Atkins-Shotgun-13_jpg.rf.a47d8eb1a5050c2509742746cfad173c.txt: Cropped images saved.
P

splitted images randomlay

In [3]:
import splitfolders

# Define the path to your image folder
input_folder = "/home/research/Kilsar_Sentinal/EfficientNet data/"

# Define the output folder where the train/val folders will be created
output_folder = "/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data"

# Split the images into train and val with an 80% - 20% ratio
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.8, .2))


Copying files: 9996 files [00:00, 19383.41 files/s]
